### Importing required packages

In [1]:
import pandas as pd
import numpy as np

### Reading the csv data files as pandas dataframes

In [2]:
persons_df = pd.read_csv('persons_data.csv')
inventory_df = pd.read_csv('inventory_data.csv')
majors_df = pd.read_csv('majors_data.csv')
occupancy_df = pd.read_csv('occupancy_data.csv')

### Checking the columns present in the persons_df

In [3]:
persons_df.columns

Index(['personId', 'firstName', 'lastName', 'email', 'dob', 'address',
       'majors'],
      dtype='object')

### Combining the first and last name of the person to create a new column containing the full name and extracting the state and city from the address and creating separate columns for the same.

In [4]:
persons_df['name'] = persons_df['firstName'] + " " + persons_df['lastName']
persons_df[['city','state']] = persons_df["address"].str.extract(r',\s*([^,]+),\s*([A-Z]{2})$')

### Removing duplicate email ids and persons id from the persons data

In [5]:
persons_df = persons_df.drop_duplicates(subset=['email'], keep='first')
persons_df = persons_df.drop_duplicates(subset=['personId'], keep='first')
persons_df = persons_df.reset_index(drop=True)

### Removing duplicate majors in majors data to save the first instance of the major id

In [6]:
majors_df = majors_df.drop_duplicates(subset=['name'], keep='first')
majors_df = majors_df.reset_index(drop=True)

### Fetching majors ID

In [7]:
#Split the majors using comma
split_majors = persons_df['majors'].str.split(', ',expand=True)

#Get the personId with the delimited majors
persons_major = pd.concat([persons_df['personId'], split_majors], axis=1)

#Change the name of the columns to major_1, major_2, ...
persons_major.columns = ['personId'] + [f'major_{i+1}' for i in range(split_majors.shape[1])]

count_cols = len(persons_major.columns)

#Unpivot the majors column
persons_major = persons_major.melt(id_vars=['personId'], value_vars=[f'major_{i+1}' for i in range(split_majors.shape[1])],
                         var_name='major_num', value_name='major')

#Dropping the rows with null major names
persons_major = persons_major.dropna(subset=['major'])

#Join with the major df to get the major ID
persons_major = persons_major.merge(majors_df, left_on='major', right_on='name', how='left')[['personId','major_num','major','id']]

#Concat all the major ID by grouping on person ID 
persons_major = persons_major.groupby(['personId']).agg({'id': lambda x: ', '.join(x)}).reset_index()

### Fetch Bed ID

In [8]:
#Fetch bed ID to occupancy data join with inventory df on basis of buildingName, roomName, bedName
person_bed = occupancy_df.merge(inventory_df, on=['buildingName','roomName','bedName'], how='left')[['personId','bedId']]

#Get Bed ID to the final df
final_df = persons_df.merge(persons_major, on='personId', how='left')
final_df = final_df.merge(person_bed, on='personId', how='left')
final_df.rename(columns={'id': "majorIds"}, inplace=True)

### There are no zip codes in the data, filling the empty 'zip' column with nan in the final output

In [9]:
final_df['zip'] = np.nan

### Defining a function to check if the address is valid or not ???

In [10]:
import re

def is_valid_address(address_string):
    city_regex = r'[A-Za-z\s]+'
    state_regex = r'[A-Z]{2}'
    postal_code_regex = r'^\d{5}$'
    
    try:
        parts = address_string.split(',')
        
        if len(parts) != 3:
            return False

        street_part = parts[0].strip()
        city_part = parts[1].strip()
        state_part = parts[2].strip()
        state_part, postal_part = state_part.split(' ')
        city_match = re.match(city_regex, city_part)
        state_match = re.match(state_regex, state_part)
        postal_code_match = re.match(postal_code_regex, postal_part)

        if city_match and state_match and postal_code_match:
            return True
        else:
            return False
    except:
        return False

final_df['valid_address'] = final_df['address'].apply(is_valid_address)

### Only filling the address column if the address is valid else with nan

In [11]:
final_df['address1'] = np.where(final_df['valid_address'], final_df['address'], np.nan)
final_df['address2'] = np.nan

### As we can see all the zip values are nan hence there are no valid addresses

In [12]:
final_df

,personId,firstName,lastName,email,dob,address,majors,name,city,state,majorIds,bedId,zip,valid_address,address1,address2
0,7b1fb398-72d7-421e-808c-631a427bfbc4,David,Taylor,david.taylor@example.com,8/15/92,"767 Walnut St, Georgetown, OH",Communications,David Taylor,Georgetown,OH,2c6e4248-da4d-4cb3-ac58-3cd04aa203fa,NaN,NaN,False,NaN,NaN
1,99a4b233-2804-420b-b757-039f0f3cceb7,Jane,Wilson,jane.wilson@example.com,8/20/69,"168 Birch St, Riverside, FL","Music, Finance",Jane Wilson,Riverside,FL,"e00c66ff-cb22-45b1-a5e8-8b3cde249016, 49f57722...",NaN,NaN,False,NaN,NaN
2,cc3b2df2-a593-493e-af82-1f0d11c7439b,John,Brown,john.brown@example.com,12/14/67,"726 Maple St, Fairview, PA",NaN,John Brown,Fairview,PA,NaN,a32b34cf-0356-4739-ab43-4a9ed70818f8,NaN,False,NaN,NaN
3,cb669177-4020-4284-a3dc-9a0c312411d4,Linda,Taylor,linda.taylor@example.com,12/4/83,"171 Spruce St, Riverside, TX","Social Work, Anthropology",Linda Taylor,Riverside,TX,"8402b4cd-a049-4ae7-8649-16af0c4c8c62, 37b765f0...",01f250f6-a3a3-492b-ba52-3f53f4483316,NaN,False,NaN,NaN
4,ba1443a2-dfeb-441b-b41d-078b36416704,Sarah,Doe,sarah.doe@example.com,8/31/86,"987 Walnut St, Georgetown, OH","Psychology, Criminal Justice, History, Foreign...",Sarah Doe,Georgetown,OH,"4d0b9e50-43d1-4da0-af2b-97d510bab9e2, b43b2475...",f8ef5ea9-e2bf-4eda-9d7a-c7cdf30dc649,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2045e65b-0845-4d48-88f3-19c708a836cf,Michael,Davis,michael.davis@example.com,4/24/91,"462 Walnut St, Madison, IL","Sociology, Economics, Computer Science",Michael Davis,Madison,IL,"dda33b53-0fff-4371-8a45-35a2192e86e8, 2c4e188e...",NaN,NaN,False,NaN,NaN
96,c4d33385-848f-42f6-843b-220fa0b0f8fc,Michael,Williams,michael.williams@example.com,8/8/94,"847 Walnut St, Madison, TX","Urban Planning, Architecture",Michael Williams,Madison,TX,"fa7f1f45-106f-4e8d-bd8c-cce0fd276256, e0bad25d...",3a0cd40a-bd21-4359-9b36-c4bf30380f9b,NaN,False,NaN,NaN
97,c4a82bb0-ba9d-446f-a121-ba7d2996c39e,Jane,Taylor,jane.taylor@example.com,5/29/55,"766 Cedar St, Madison, FL","Sociology, Biology, Agriculture, Art",Jane Taylor,Madison,FL,"dda33b53-0fff-4371-8a45-35a2192e86e8, b33ae65d...",NaN,NaN,False,NaN,NaN
98,0a10e180-a33a-4d04-8519-ae951f56b478,Sarah,Johnson,sarah.johnson@example.com,12/18/99,"719 Maple St, Madison, CA","Physics, Biology, History, Social Work, Mathem...",Sarah Johnson,Madison,CA,"9dd4dfda-b0d2-4d4a-8642-a44db00f4a15, b33ae65d...",NaN,NaN,False,NaN,NaN


In [13]:
#Convert dob to pd datetime, this may give results like 55 to 2055 instead of 1955
final_df['dob'] = pd.to_datetime(final_df['dob'], format="%m/%d/%y")

#To fix the above problem we check if year is beyond 2024 then subtract 100 years from the dob to get desired result
final_df['dob'] = final_df['dob'].apply(lambda x: x if x.year <= 2024 else x.replace(year=x.year - 100))

In [14]:
final_df[['personId','name','email','dob','address1','address2','city','state','zip','majorIds','bedId']]\
.to_csv('question2_solution.csv', index=False)